In [19]:
!pip install gurobipy

In [20]:
import numpy as np
import pandas as pd
import math
import time
from gurobipy import *


In [21]:
df_demands = pd.read_excel("Spring25Project_Data.xlsx", sheet_name='demands')
df_coordinates = pd.read_excel("Spring25Project_Data.xlsx", sheet_name='coordinates')

coordinates = df_coordinates[['x-coordinate', 'y-coordinate']].values
demands = df_demands['Demand'].values


In [22]:
mean_lat = df_coordinates["y-coordinate"].mean()
meters_per_deg_lat = 111320
meters_per_deg_lon = 111320 * math.cos(math.radians(mean_lat))
df_coordinates["x_m"] = (df_coordinates["x-coordinate"] - df_coordinates["x-coordinate"].min()) * meters_per_deg_lon
df_coordinates["y_m"] = (df_coordinates["y-coordinate"] - df_coordinates["y-coordinate"].min()) * meters_per_deg_lat
coords_m = df_coordinates[['x_m', 'y_m']].values

In [23]:
import math

lat = df_coordinates["y-coordinate"]
lon = df_coordinates["x-coordinate"]

# Calculate the average latitude
mean_lat = lat.mean()

# 1 degree of latitude is approximately 111320 meters
meters_per_deg_lat = 111320

# 1 degree of longitude is approximately 111320 * cos(average latitude) meters
meters_per_deg_lon = 111320 * math.cos(math.radians(mean_lat))

# Transform coordinates relative to a reference point
df_coordinates["x_m"] = (lon - lon.min()) * meters_per_deg_lon
df_coordinates["y_m"] = (lat - lat.min()) * meters_per_deg_lat

# Convert the transformed coordinates (in meters) to a numpy array
coords_m = df_coordinates[['x_m', 'y_m']].values

# Compute the distance matrix using numpy broadcasting
d = np.sqrt(((coords_m[:, None, :] - coords_m[None, :, :]) ** 2).sum(axis=2))



In [24]:
d = np.sqrt(((coords_m[:, None, :] - coords_m[None, :, :]) ** 2).sum(axis=2))
dist = d.tolist()

In [25]:
nodes = list(range(len(coords_m)))
demand = demands
veh_cap = 85
k = math.ceil(sum(demand) / veh_cap)

print(f"Total demand: {sum(demand)}")
print(f"Vehicle capacity: {veh_cap}")
print(f"Estimated number of vehicles: {k}")


Total demand: 526
Vehicle capacity: 85
Estimated number of vehicles: 7


In [26]:
# Create model
vrp = Model("vrp_model")

Restricted license - for non-production use only - expires 2026-11-23


In [27]:
#Create variables
x = vrp.addVars(nodes, nodes, vtype=GRB.BINARY, name="x")
u = vrp.addVars(nodes, vtype=GRB.INTEGER, name="u")

In [28]:
#Add constraint
vrp.addConstrs(((quicksum(x[i,j] for j in nodes if j != i) == 1) for i in nodes if i != 0), name = '1'); #outflow
vrp.addConstrs(((quicksum(x[i,j] for i in nodes if i != j) == 1) for j in nodes if j != 0), name = '2'); #inflow
vrp.addConstr(((quicksum(x[0,j] for j in nodes) == k)), name = '3'); #depot out
vrp.addConstr(((quicksum(x[i,0] for i in nodes) == k)), name = '4'); #depot in
vrp.addConstrs(((u[i] - u[j] + veh_cap*x[i,j] <= veh_cap - demand[j]) for i in nodes for j in nodes if i!=j and i != 0 and j != 0), name = '5'); #mtz subtour
vrp.addConstrs(((u[i] <= veh_cap) for i in nodes), name = '6'); #max cap
vrp.addConstrs(((u[i] >= demand[i]) for i in nodes), name = '7'); #min demand

In [29]:
# Set objective function
vrp.setObjective(quicksum(dist[i][j] * x[i,j] for i in nodes for j in nodes if i != j), GRB.MINIMIZE)
vrp.setParam("TimeLimit", 900) #time limit for 15minutes

Set parameter TimeLimit to value 900


In [30]:
# Solve
start_time = time.time()
vrp.optimize()
runtime_gurobi = time.time() - start_time

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Non-default parameters:
TimeLimit  900

Optimize a model with 1562 rows, 1560 columns and 7262 nonzeros
Model fingerprint: 0x790f4df0
Variable types: 0 continuous, 1560 integer (1521 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [4e+05, 1e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+01]
Presolve removed 110 rows and 71 columns
Presolve time: 0.04s
Presolved: 1452 rows, 1489 columns, 7024 nonzeros
Variable types: 0 continuous, 1489 integer (1451 binary)

Root relaxation: objective 5.025557e+07, 144 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

    

In [31]:
# Solution
solution = []
if vrp.status == GRB.OPTIMAL or vrp.status == GRB.TIME_LIMIT:
    selected = [(i, j) for i in nodes for j in nodes if i != j and x[i,j].X > 0.5]
    routes = []
    while selected:
        route = [0]
        while True:
            for i, j in selected:
                if i == route[-1]:
                    route.append(j)
                    selected.remove((i, j))
                    break
            else:
                break
        if route[-1] == 0:
            routes.append(route)

    print("\nOptimal routes:")
    for r in routes:
        print(r)
    print(f"\nTotal cost: {vrp.objVal:.2f}")
    print(f"Runtime: {runtime_gurobi:.4f} seconds")
else:
    print("No feasible solution found within the time limit.")



Optimal routes:
[0, 10, 16, 4, 2, 37, 0, 11, 36, 17, 23, 1, 6, 0, 13, 0, 24, 12, 38, 3, 15, 0, 26, 5, 9, 28, 29, 0, 30, 20, 32, 27, 34, 22, 18, 21, 0, 31, 14, 35, 25, 33, 19, 8, 7, 0]

Total cost: 86527607.39
Runtime: 900.0464 seconds


In [76]:
print("\n Final Gurobi Summary for Part C:")
print("{:<45} {:<12} {:<12}".format("Method", "Cost", "Time (s)"))
print("_" * 70)
print("{:<45} {:<12.2f} {:<12.4f}".format("Gurobi Optimal VRP Solution", vrp.objVal, runtime_gurobi))



 Final Gurobi Summary for Part C:
Method                                        Cost         Time (s)    
______________________________________________________________________
Gurobi Optimal VRP Solution                   86527607.39  900.0464    


# **Final Comparison**

The "% vs Gurobi" column indicates how much more expensive each method is compared to the Gurobi optimal solution.
Lower values mean the method is closer to optimal.
(For more info VRP_Final_Comparison.xlsx)


In [83]:
#Load final comparison table from Excel file
file_path = "VRP_Final_Comparison.xlsx"
comparison_df = pd.read_excel(file_path)
print("\nFinal Comparison Table from VRP_Final_Comparison.xlsx:")
print(comparison_df)


Final Comparison Table from VRP_Final_Comparison.xlsx:
                            Method          Cost  % Improve  Time (s)  \
0                CFRS + NN + 2-opt  1.115669e+08       0.32    0.4559   
1           CFRS + NN + Relocation  1.062439e+08       5.07    0.4284   
2   RFCS + Christofides + Relocate  1.039499e+08       2.04    0.0159   
3  RFCS + Nearest Neighbor + 2-opt  1.009624e+08       0.18    0.0039   
4      Gurobi Optimal VRP Solution  8.652761e+07        NaN  900.0464   

   % vs Gurobi  
0    22.443314  
1    18.557614  
2    16.760292  
3    14.297183  
4     0.000000  


/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
